In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [2]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        # return score, tag_seq
        return tag_seq

In [3]:
# data
import numpy as np
import pandas as pd

PARTICIPANT_ID = 1 #4348

df_csv=pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_wrong.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
df_csv=df_csv[(df_csv['participant_ID']==PARTICIPANT_ID)] # take 1st participant data
df_csv=df_csv.drop(['participant_ID'],axis=1)

In [4]:
q_index_list = sorted(list(set(df_csv['Q_ID'])))

total_q_list = []
for q in q_index_list:
    df_q = df_csv[df_csv['Q_ID'] == q]
    text_list = list(df_q['text'])
    label_list = list(df_q['fixation1'])
    q_list = [text_list, label_list]
    total_q_list.append(q_list)

print(
    '\n',total_q_list[0], # first data with text and label
    '\n',total_q_list[0][0], # first data's text
    '\n',total_q_list[0][1], # first data's label
)



 [['兩', '個', '量筒', '各', '裝', '50', 'cc', '水', '與', '50', 'cc', '酒精', '混合', '後', '為', '多少', 'cc', '等於', '100', 'cc', '少於', '100', 'cc', '接近', '100', 'cc'], [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]] 
 ['兩', '個', '量筒', '各', '裝', '50', 'cc', '水', '與', '50', 'cc', '酒精', '混合', '後', '為', '多少', 'cc', '等於', '100', 'cc', '少於', '100', 'cc', '接近', '100', 'cc'] 
 [0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]


In [5]:
# word_to_ix 
# unique word and its representing number
# make it all word word_to_ix

word_index_df = pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_data.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
word_index_df = word_index_df[(word_index_df['participant_ID'] == 1)]
word_index_list = sorted(list(set(word_index_df['Q_ID'])))

total_word_index_list = []
for q in word_index_list:
    df_q = word_index_df[word_index_df['Q_ID'] == q]
    text_list = list(df_q['text'])
    label_list = list(df_q['fixation1'])
    q_list = [text_list, label_list]
    total_word_index_list.append(q_list)


word_to_ix = {}
for sentence, tags in total_word_index_list:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_to_ix

{'兩': 0,
 '個': 1,
 '量筒': 2,
 '各': 3,
 '裝': 4,
 '50': 5,
 'cc': 6,
 '水': 7,
 '與': 8,
 '酒精': 9,
 '混合': 10,
 '後': 11,
 '為': 12,
 '多少': 13,
 '等於': 14,
 '100': 15,
 '少於': 16,
 '接近': 17,
 '沙罐': 18,
 '置入': 19,
 '不同': 20,
 '濃度': 21,
 '鹽水': 22,
 '中': 23,
 '所': 24,
 '受': 25,
 '浮力': 26,
 '會': 27,
 '如何': 28,
 '相同': 29,
 '愈': 30,
 '高': 31,
 '大': 32,
 '小': 33,
 '將': 34,
 '幼苗': 35,
 '平放': 36,
 '上側': 37,
 '或是': 38,
 '下側': 39,
 '生長素': 40,
 '較': 41,
 '使': 42,
 '莖': 43,
 '向': 44,
 '上': 45,
 '長': 46,
 '一樣': 47,
 '多': 48,
 '外加': 49,
 '電池': 50,
 '碳棒': 51,
 '及': 52,
 '銅棒': 53,
 '放入': 54,
 '硫酸銅': 55,
 '𥚃': 56,
 '發生': 57,
 '什麼': 58,
 '反應': 59,
 '電解': 60,
 '氧化還原': 61,
 '以': 62,
 '針筒': 63,
 '吸取': 64,
 '熱水': 65,
 '並': 66,
 '封住': 67,
 '針頭': 68,
 '活塞': 69,
 '外': 70,
 '拉出': 71,
 '裡面': 72,
 '的': 73,
 '有': 74,
 '何': 75,
 '現象': 76,
 '水位': 77,
 '不變': 78,
 '沸騰': 79,
 '四散': 80,
 '處': 81,
 '把': 82,
 '支': 83,
 '長度': 84,
 '蠟燭': 85,
 '點燃': 86,
 '用': 87,
 '一': 88,
 '透明': 89,
 '玻璃杯': 90,
 '蓋住': 91,
 '哪': 92,
 '先': 93,
 '熄滅': 94

In [6]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 4 #5
HIDDEN_DIM = 4 #4


training_data = total_q_list#[:37]
#testing_data = total_q_list[37:]

tag_to_ix = {0: 0, 1: 1, START_TAG: 2, STOP_TAG: 3}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print('real label：'+ str(precheck_tags.tolist()))
    print('untrained label prediction：'+ str(model(precheck_sent)))

print('=============start BiLSTM+CRF model training=============')
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(200):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
    if epoch%50 == 0:
        print(f'the{epoch}th epoch Loss:{loss[0]}')


output_path = 'right_1_trained_model.cpt'
torch.save(model, output_path)
print('=============trained model saved=============\n\n')



print('=============load trained model and predict=============')
model_path = 'right_1_trained_model.cpt'
trained_ner_model = torch.load(model_path)
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print('trained label prediction：' + str(trained_ner_model(precheck_sent)))

# time spent : 3min

real label：[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]
untrained label prediction：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
=============start BiLSTM+CRF model training=============
the0th epoch Loss:13.505768775939941
the50th epoch Loss:11.856651306152344
the100th epoch Loss:8.141481399536133
the150th epoch Loss:3.1117677688598633
=============trained model saved=============


=============load trained model and predict=============
trained label prediction：[0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0]


In [10]:
# # accuracy from scratch
# total_wrong = []
# acccuracy_score = []
# for q in range(47):
#     precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
#     predict = model(precheck_sent)
#     answer = training_data[q][1]
#     accuracy = sum([1 for i,j in zip(predict,answer) if i==j])/len(answer)
#     print(str(q+1) + " accuracy:", round(accuracy,2))
#     acccuracy_score.append(accuracy)
#     wrong_list = [i for i,j in zip(predict,answer) if i!=j] # wrong answer
#     total_wrong.append(wrong_list)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# training predict score
total_result_list = []
total_output_list = []
for q in range(len(training_data)):
    precheck_sent = prepare_sequence(training_data[q][0], word_to_ix)
    predict = trained_ner_model(precheck_sent)
    answer = training_data[q][1]
    output_list = [training_data[q][0], predict, answer]
    total_output_list.append(output_list)
    
    accuracy = accuracy_score(answer, predict)
    precision = precision_score(answer, predict)
    recall = recall_score(answer, predict)
    f1score = f1_score(answer, predict)
    result_list = [accuracy, precision, recall, f1score]
    result_list = [round(elem, 2) for elem in result_list]
    total_result_list.append(result_list)

In [8]:
total_qresult_df = pd.DataFrame()
for q in range(len(total_output_list)):
    q_result = pd.DataFrame(total_output_list[q])
    total_qresult_df = pd.concat([total_qresult_df, q_result],axis=0)
total_qresult_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/wrong_1_qresult.csv")
total_qresult_df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,兩,個,量筒,各,裝,50,cc,水,與,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,1,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,1,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,將,幼苗,平放,幼苗,上側,或是,下側,生長素,濃度,較,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0,1,1,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,將,硫酸鈉,溶液,加熱,時,燒杯,內,硫酸鈉,的,溶解度,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,1,1,0,0,0,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
total_result_df = pd.DataFrame(total_result_list)
total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']

total_result_df.loc[total_result_df.shape[0]] = total_result_df.mean(axis=0)
total_result_df = total_result_df.rename(index={total_result_df.shape[0]-1: 'mean'})
total_result_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/wrong_1_score.csv")
total_result_df

,accuracy,precision,recall,f1score
0,1.000000,1.00000,1.000000,1.000000
1,1.000000,1.00000,1.000000,1.000000
2,1.000000,1.00000,1.000000,1.000000
3,0.810000,0.88000,0.780000,0.820000
4,0.880000,0.83000,0.940000,0.880000
5,1.000000,1.00000,1.000000,1.000000
6,1.000000,1.00000,1.000000,1.000000
7,0.920000,1.00000,0.790000,0.880000
8,0.950000,0.89000,1.000000,0.940000
9,0.950000,0.90000,1.000000,0.950000


---
---
---

# right model on wrong data prediction

In [10]:
# wrong data
wrong_df_csv = pd.read_csv('/Users/apple/Desktop/deep_eye/dataset/input_right.csv',usecols=['participant_ID','Q_ID','text','fixation1'])
wrong_df_csv = wrong_df_csv[(wrong_df_csv['participant_ID']==PARTICIPANT_ID)] # take 1st participant data
wrong_df_csv = wrong_df_csv.drop(['participant_ID'],axis=1)

In [11]:
wrong_q_index_list = sorted(list(set(wrong_df_csv['Q_ID'])))

wrong_total_q_list = []
for q in wrong_q_index_list:
    df_q = wrong_df_csv[wrong_df_csv['Q_ID'] == q]
    text_list = list(df_q['text'])
    label_list = list(df_q['fixation1'])
    q_list = [text_list, label_list]
    wrong_total_q_list.append(q_list)

print(
    '\n',wrong_total_q_list[0], # first data with text and label
    '\n',wrong_total_q_list[0][0], # first data's text
    '\n',wrong_total_q_list[0][1], # first data's label
)



 [['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小'], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]] 
 ['沙罐', '置入', '不同', '濃度', '鹽水', '中', '所', '受', '浮力', '會', '如何', '相同', '濃度', '愈', '高', '所', '受', '浮力', '愈', '大', '濃度', '愈', '高', '所', '受', '浮力', '愈', '小'] 
 [0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]


In [12]:
print(len(wrong_total_q_list))

18


In [13]:
# predict right questions

wrong_total_result_list = []
wrong_total_output_list = []
for q in range(len(wrong_total_q_list)): # range(len(wrong_total_q_list))
    wrong_precheck_sent = prepare_sequence(wrong_total_q_list[q][0], word_to_ix)
    wrong_predict = trained_ner_model(wrong_precheck_sent)
    wrong_answer = wrong_total_q_list[q][1]
    wrong_output_list = [wrong_total_q_list[q][0], wrong_predict, wrong_answer]
    wrong_total_output_list.append(wrong_output_list)
    
    accuracy = accuracy_score(wrong_answer, wrong_predict)
    precision = precision_score(wrong_answer, wrong_predict)
    recall = recall_score(wrong_answer, wrong_predict)
    f1score = f1_score(wrong_answer, wrong_predict)
    wrong_result_list = [accuracy, precision, recall, f1score]
    wrong_result_list = [round(elem, 2) for elem in wrong_result_list]
    wrong_total_result_list.append(wrong_result_list)

In [14]:
wrong_total_qresult_df = pd.DataFrame()
for q in range(len(wrong_total_output_list)):
    wrong_q_result = pd.DataFrame(wrong_total_output_list[q])
    wrong_total_qresult_df = pd.concat([wrong_total_qresult_df, wrong_q_result],axis=0)
wrong_total_qresult_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/wrong_on_right_1_qresult.csv")
wrong_total_qresult_df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,沙罐,置入,不同,濃度,鹽水,中,所,受,浮力,會,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,1,0,1,0,0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,1,1,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,外加,電池,將,碳棒,及,銅棒,放入,硫酸銅,𥚃,將,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,1,1,1,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,1,0,0,1,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,以,針筒,吸取,熱水,並,封住,針頭,將,針頭,活塞,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,1,0,1,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0,0,1,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,雌蕊,的,花柱,內,有,多少,個,花粉管,一,個,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
wrong_total_result_df = pd.DataFrame(wrong_total_result_list)
wrong_total_result_df.columns = ['accuracy', 'precision', 'recall', 'f1score']

wrong_total_result_df.loc[wrong_total_result_df.shape[0]] = wrong_total_result_df.mean(axis=0)
wrong_total_result_df = wrong_total_result_df.rename(index={wrong_total_result_df.shape[0]-1: 'mean'})
wrong_total_result_df.to_csv("/Users/apple/Desktop/deep_eye/dataset/wrong_on_right_1_score.csv")
wrong_total_result_df

,accuracy,precision,recall,f1score
0,0.680,0.750000,0.710000,0.730000
1,0.680,0.800000,0.670000,0.730000
2,0.600,0.530000,0.620000,0.570000
3,0.500,0.250000,0.200000,0.220000
4,0.650,0.750000,0.200000,0.320000
5,0.480,0.500000,0.210000,0.300000
6,0.420,0.120000,0.110000,0.120000
7,0.440,0.400000,0.250000,0.310000
8,0.540,0.400000,0.400000,0.400000
9,0.440,0.370000,0.770000,0.500000


---
---
---

In [18]:
# # draw confusion matrix
# import matplotlib.pyplot as plt
# conf_matrix = confusion_matrix(y_true=answer, y_pred=predict)
# fig, ax = plt.subplots(figsize=(5, 5))
# ax.matshow(conf_matrix, cmap=plt.cm.Oranges, alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()